# Advent of code 2023

## [Day 3](https://adventofcode.com/2023/day/3): Gear Ratios

### Connecting to Neo4j database *via* `graphdatascience` python client

In [ ]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

### Ingesting data

In [ ]:
file = open('input.txt', 'r')
data = []
for ix_row, line in enumerate(file):
    for ix_col, symbol in enumerate(line):
        if symbol in ['\n', '.']:
            continue
        data.append({"row":ix_row, "col":ix_col, "symbol":symbol})

query ="""
UNWIND $data AS c_data
CALL {
    WITH c_data
    CREATE (:Cell {row:c_data.row, col:c_data.col, symbol:c_data.symbol})
} IN TRANSACTIONS OF 5000 ROWS
"""
gds.run_cypher(query, {"data":data})

### Adding `Symbol` label to non-digit `Cell`s

In [ ]:
gds.run_cypher("CREATE INDEX cell_row_col IF NOT EXISTS FOR (c:Cell) ON (c.row, c.col)")
gds.run_cypher("""
MATCH (c:Cell)
WHERE NOT "0123456789" CONTAINS c.symbol
SET c:Symbol
""")

### Creating `NEXT` relationship between digit `Cell`s of a same part number

In [ ]:
query = """
MATCH (c:Cell&!Symbol)
CALL{
    WITH c
    MATCH (x:Cell&!Symbol {row:c.row, col: c.col +1})
    MERGE (c)-[:NEXT]->(x)
    } IN TRANSACTIONS OF 5000 ROWS
"""
gds.run_cypher(query)

### Creating `NEAR` relationship between `Symbol` and adjacent digit `Cell`

In [ ]:
query = """
MATCH (s:Cell:Symbol)
CALL{
    WITH s
    MATCH (x:Cell&!Symbol)
    WHERE abs(x.row-s.row) <=1 AND abs(x.col-s.col) <=1
    MERGE (s)-[:NEAR]->(x)
    } IN TRANSACTIONS OF 5000 ROWS
"""
gds.run_cypher(query)

### Computing `partNumber` when necessary

In [ ]:
gds.run_cypher("""
MATCH (first:Cell&!Symbol WHERE NOT EXISTS {(:Cell&!Symbol)-[:NEXT]->(first)})
    (()-[:NEXT]->(followings:Cell&!Symbol))*
    (last:Cell&!Symbol WHERE NOT EXISTS {(last)-[:NEXT]->(:Cell&!Symbol)})
WHERE any(cell IN [first]+followings WHERE (cell)--(:Symbol))
WITH [first]+followings AS nodes, toInteger(apoc.text.join([cell IN [first]+followings | cell.symbol],"")) AS part_number
UNWIND nodes AS n
SET n.partNumber = part_number
""")

### Part 1 solution

In [ ]:
gds.run_cypher("""
MATCH (first:Cell&!Symbol WHERE NOT EXISTS {(:Cell&!Symbol)-[:NEXT]->(first)} AND first.partNumber IS NOT NULL)
RETURN sum(first.partNumber) AS part1
""")

### Part 2 solution

In [ ]:
gds.run_cypher("""
MATCH (s:Symbol {symbol:'*'})--(p:Cell&!Symbol)
WITH s, apoc.coll.toSet(collect(p.partNumber)) AS partNumbers
WHERE size(partNumbers) = 2
RETURN sum(partNumbers[0]*partNumbers[1]) AS part2
""")